In [83]:
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import os
import time
import tensorflow as tf

from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping
from keras import backend as K
from keras.preprocessing import image                  
from sklearn.utils import shuffle
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Flatten, Dense
from keras.models import Sequential, Model
from keras.layers import BatchNormalization
from keras import regularizers, applications, optimizers, initializers
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.applications import EfficientNetB6
from tensorflow.keras.applications import EfficientNetB7

In [35]:
def binary_accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)))

def precision_threshold(threshold = 0.5):
    def precision(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(y_pred)
        precision_ratio = true_positives / (predicted_positives + K.epsilon())
        return precision_ratio
    return precision

def recall_threshold(threshold = 0.5):
    def recall(y_true, y_pred):
        threshold_value = threshold
        y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold_value), K.floatx())
        true_positives = K.round(K.sum(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.clip(y_true, 0, 1))
        recall_ratio = true_positives / (possible_positives + K.epsilon())
        return recall_ratio
    return recall

def fbeta_score_threshold(beta = 1, threshold = 0.5):
    def fbeta_score(y_true, y_pred):
        threshold_value = threshold
        beta_value = beta
        p = precision_threshold(threshold_value)(y_true, y_pred)
        r = recall_threshold(threshold_value)(y_true, y_pred)
        bb = beta_value ** 2
        fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
        return fbeta_score
    return fbeta_score

In [ ]:
log = CSVLogger('saved_models/log_pretrained_CNN.csv')
checkpointer = ModelCheckpoint(filepath='saved_models/pretrainedVGG.best.from_scratch.hdf5', verbose=1, save_best_only=True)

In [36]:
diseases = [
    'Cardiomegaly','Emphysema','Effusion',
    'Hernia','Nodule','Pneumothorax',
    'Atelectasis','Pleural_Thickening',
    'Mass','Edema','Consolidation',
    'Infiltration','Fibrosis','Pneumonia'
]

dataset_df = pd.read_csv('./dataset_information/Data_Entry_2017.csv')

In [37]:
# Applying One Hot Encoding to Labels
for disease in diseases:
    dataset_df[disease] = dataset_df['Finding Labels'].apply(lambda x: 1 if disease in x else 0)

In [38]:
image_labels = dataset_df[diseases].to_numpy()
image_paths = {
    os.path.basename(x): x for x in glob(os.path.join('.', 'images', '*.png'))
}

print(f"Samples Found: {len(image_paths)}")

Samples Found: 112120


In [39]:
# Storing path to each image against image name in the dataframe
dataset_df['Image Path'] = dataset_df['Image Index'].map(image_paths.get)

In [40]:
images_list = dataset_df['Image Path'].tolist()

labelB = (dataset_df[diseases].sum(axis = 1) > 0).tolist()
labelB = np.array(labelB, dtype = int)

In [41]:
def read_image_to_tensor(path, shape):
    # Loads RGB image to PIL format
    img = image.load_img(path, target_size = shape)
    
    # Convert PIL image to 3D tensor of specific shape
    # and normalizes it by dividing each pixel by 255
    normalized_image_tensor = image.img_to_array(img) / 255
    
    # Convert 3D tensor to 4D tensor with specific shape 
    # (1, shape, 3) and return it
    return np.expand_dims(normalized_image_tensor, axis = 0)

In [42]:
def image_to_array(paths, shape):
    images_arrays = [read_image_to_tensor(path, shape) for path in tqdm(paths, desc = "Progress", ncols = 100)]
    return np.vstack(images_arrays)

In [43]:
IMAGE_SHAPE = (64, 64)

In [44]:
# Samples in Training Set: 70%
# Samples in Validation Set: 8%
# Samples in Testing Set: 22%

# Storing labels of samples for each set
train_labels = labelB[ : 85000][ : , np.newaxis]
valid_labels = labelB[85000 : 95000][ : , np.newaxis]
test_labels = labelB[95000 : ][ : , np.newaxis]

# Storing arrays of samples for each set
training_samples = image_to_array(images_list[ : 85000], shape = IMAGE_SHAPE)
validation_samples = image_to_array(images_list[85000 : 95000], shape = IMAGE_SHAPE)
test_samples = image_to_array(images_list[95000 : ], shape = IMAGE_SHAPE)

Progress: 100%|█████████████████████████████████████████████████| 3500/3500 [00:47<00:00, 73.22it/s]


In [84]:
# Creating a model with EfficientNet B0 as base.
efficient_net_b0 = EfficientNetB0(
    weights = 'imagenet',
    include_top = False,
    input_shape = training_samples.shape[1 : ]
)

custom_classifier = Sequential()
custom_classifier.add(GlobalAveragePooling2D(input_shape = efficient_net_b0.output_shape[1 : ]))
custom_classifier.add(Dropout(0.2))
custom_classifier.add(Dense(256, activation = 'relu'))
custom_classifier.add(Dropout(0.2))
custom_classifier.add(Dense(512, activation = 'relu'))
custom_classifier.add(Dropout(0.2))
custom_classifier.add(Dense(256, activation = 'relu'))
custom_classifier.add(Dropout(0.2))
custom_classifier.add(Dense(50, activation = 'relu'))
custom_classifier.add(Dropout(0.2))
custom_classifier.add(Dense(1, activation = 'sigmoid'))

model = Model(inputs = efficient_net_b0.input, outputs = custom_classifier(efficient_net_b0.output))
# model.summary()

27033600/27018416 [==============================] - 29s 1us/step


In [85]:
# Defining 2 optimizers to test the model with.

SGD_optimizer = tf.keras.optimizers.SGD(
    learning_rate = 1e-4, 
    decay = 1e-6, 
    momentum = 0.9, 
    nesterov = True
)
adam_optimizer = tf.keras.optimizers.Adam(
    learning_rate = 0.001,
    beta_1 = 0.9,
    beta_2 = 0.999,
)

In [86]:
# Defining object for augmentation 

train_datagen = ImageDataGenerator(
    featurewise_center = False,  # set input mean to 0 over the dataset
    samplewise_center = False,  # set each sample mean to 0
    featurewise_std_normalization = False,  # divide inputs by std of the dataset
    samplewise_std_normalization = False,  # divide each input by its std
    zca_whitening = False,  # apply ZCA whitening
    rotation_range = 10,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range = 0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range = 0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip = True,  # randomly flip images
    vertical_flip = False 
)

In [87]:
# Defining some hyper-parameters

EPOCHS = 5
BATCH_SIZE = 32

In [88]:
# Compiling model with loss function, optimizer and metrics

model.compile(
    optimizer = SGD_optimizer,
    loss = 'binary_crossentropy',
    metrics = [
        'accuracy',
        precision_threshold(threshold = 0.5), 
        recall_threshold(threshold = 0.5), 
        fbeta_score_threshold(beta=0.5, threshold = 0.5)
    ]
)

In [89]:
%%timeit -n1 -r1

history = model.fit_generator(
    train_datagen.flow(
        training_samples, 
        train_labels, 
        batch_size = BATCH_SIZE
    ),
    steps_per_epoch = len(training_samples) // BATCH_SIZE,
    validation_data = (validation_samples, valid_labels),
    validation_steps = len(validation_samples) // BATCH_SIZE,
    epochs = EPOCHS,
    callbacks = [checkpointer, log], 
    verbose = 1
)

Epoch 1/5
234/234 [==============================] - 44s 137ms/step - loss: 0.7006 - accuracy: 0.5281 - precision: 0.4490 - recall: 0.3352 - fbeta_score: 0.4089 - val_loss: 0.6917 - val_accuracy: 0.5150 - val_precision: 0.4137 - val_recall: 0.3479 - val_fbeta_score: 0.3771

Epoch 00001: val_loss did not improve from 0.67779
Epoch 2/5
234/234 [==============================] - 29s 126ms/step - loss: 0.6985 - accuracy: 0.5359 - precision: 0.4399 - recall: 0.2417 - fbeta_score: 0.3664 - val_loss: 0.7002 - val_accuracy: 0.4410 - val_precision: 0.4004 - val_recall: 0.7430 - val_fbeta_score: 0.4358

Epoch 00002: val_loss did not improve from 0.67779
Epoch 3/5
234/234 [==============================] - 30s 126ms/step - loss: 0.6969 - accuracy: 0.5371 - precision: 0.4360 - recall: 0.2393 - fbeta_score: 0.3635 - val_loss: 0.6873 - val_accuracy: 0.5680 - val_precision: 0.3602 - val_recall: 0.1104 - val_fbeta_score: 0.2228

Epoch 00003: val_loss did not improve from 0.67779
Epoch 4/5
234/234 [===

In [90]:
prediction = model.predict(test_samples)

In [91]:
threshold = 0.5
beta = 0.5

accuracy = K.eval(binary_accuracy(K.variable(value=test_labels), K.variable(value=prediction)))
precision = K.eval(precision_threshold(threshold = threshold)(K.variable(value=test_labels),K.variable(value=prediction)))
recall = K.eval(recall_threshold(threshold = threshold)(K.variable(value=test_labels),K.variable(value=prediction)))
f1_score = K.eval(fbeta_score_threshold(beta = beta, threshold = threshold)(K.variable(value=test_labels),K.variable(value=prediction)))

In [92]:
print (f"Accuracy: {accuracy} \nPrecision: {precision} \nRecall: {recall} \nF1-Score: {f1_score}")

Accuracy: 0.5217142701148987 
Precision: 0.3809082508087158 
Recall: 0.29004940390586853 
F1-Score: 0.3584509789943695
